In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-05-21 12:30:00--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   155MB/s    in 0.2s    

2024-05-21 12:30:00 (155 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [21]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author', 'year', 'publisher'],
    usecols=['isbn', 'title', 'author', 'year', 'publisher'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str', 'year': 'str', 'publisher': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [22]:
# Search the isbns of the books that were rated more thant 100 times
book_count = df_ratings['isbn'].value_counts()
good_books = book_count[book_count >= 100].index
#create a list of good books
good_books = df_books.loc[df_books["isbn"].isin(good_books)]

#Count the numer of ratings for each user
user_count = df_ratings['user'].value_counts()
#create a list of good users
good_users = user_count[user_count >= 200].index

#remove users and books from dataset
df = df_ratings.loc[df_ratings['user'].isin(good_users)]
df = df.loc[df['isbn'].isin(good_books['isbn'])]

#add titles to df dataset to work directly with titles instead of isbn
df = pd.merge(df, good_books, on='isbn').reset_index()

#create a table, neds to be a pivot_table to avoid index problems
rating_books_pivot = df.pivot_table(index='title', columns='user', values='rating').fillna(0)

nbrs = NearestNeighbors(metric='cosine')
nbrs.fit(rating_books_pivot)

NearestNeighbors(metric='cosine')

In [28]:
def get_recommends(title = "",n_rec=6):
  distance, indice = nbrs.kneighbors(rating_books_pivot.loc[[title]], n_neighbors=n_rec, return_distance=True)

  indice = indice[0][1:]
  distance = distance[0][1:]

  titles = [rating_books_pivot.index[i] for i in indice]
  recommended = [list(z) for z in zip(titles, distance)][::-1]

  return [title, recommended]


recommended_books=get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))",n_rec=10)

print(recommended_books)

['The Queen of the Damned (Vampire Chronicles (Paperback))', [['The Search', 0.8313657], ['Neuromancer (Remembering Tomorrow)', 0.81413937], ['The Gunslinger (The Dark Tower, Book 1)', 0.80523217], ['Lasher: Lives of the Mayfair Witches (Lives of the Mayfair Witches)', 0.79571664], ['Catch 22', 0.7939835], ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566], ['Interview with the Vampire', 0.73450685], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384], ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116]]]


In [31]:
#Use    the    cell    below    to    test    your    function.    The
#test_book_recommendation() function will inform you if you passed the
#challenge or need to keep trying.

books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
    if test_pass:
      print("You passed the challenge!")
    else:
      print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge!
You passed the challenge!
